In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from toolz.functoolz import pipe

In [ ]:
%run -i "preprocess_lang.ipynb"
%run -i "preprocess_nolang.ipynb"

In [ ]:
df = pd.read_json("dataset/jsonl/en-US.jsonl", lines=True)
df

In [ ]:
df = pipe(df, remove_punctuation, lowercase, tokenize, apply_stemming)

df["utt"] = df["utt"].apply(lambda x: " ".join(x))
df

In [ ]:
def get_corpus(df):
    return df["utt"].tolist()


get_corpus(df)

In [ ]:
print("Scenarios:")
print(df["scenario"].unique())

print("\nIntents:")
print(df["intent"].unique())

In [ ]:
fig, ax = plt.subplots()

fig.set_figwidth(20)
fig.set_dpi(800)

ax.bar(df["scenario"].unique(), df["scenario"].value_counts(), color="#9179FA")
ax.set_title("Scenario Distribution")
ax.set_xlabel("Scenario")
ax.set_ylabel("Count")

plt.style.use("dark_background")
plt.xticks(rotation=90)
plt.show()

In [ ]:
fig, ax = plt.subplots()

fig.set_figwidth(20)
fig.set_dpi(800)

ax.bar(df["intent"].unique(), df["intent"].value_counts(), color="#9179FA")
ax.set_title("Intent Distribution")
ax.set_xlabel("Intent")
ax.set_ylabel("Count")

plt.style.use("dark_background")
plt.xticks(rotation=90)
plt.show()

In [ ]:

from wordcloud import WordCloud


def create_word_cloud(content):
    wordcloud = WordCloud().generate(" ".join(content))

    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
create_word_cloud(get_corpus(df))

In [ ]:
create_word_cloud(get_corpus(df[df["scenario"] == "cooking"]))

In [ ]:
create_word_cloud(get_corpus(df[df["intent"] == "general_quirky"]))